In [1]:
# Import required libraries
from openai import OpenAI
from dotenv import load_dotenv
import os
import requests
import json

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPEN_API_KEY"))

# Weather function using RapidAPI
def get_weather(location):
    url = "https://ai-weather-by-meteosource.p.rapidapi.com/find_places"
    querystring = {"text": location, "language": "en"}
    headers = {
        "x-rapidapi-key": "cd63083a33msha2e5829aa1a8e78p183c42jsn92196e6a83c6",
        "x-rapidapi-host": "ai-weather-by-meteosource.p.rapidapi.com"
    }
    response = requests.get(url, headers=headers, params=querystring)
    return response.json()

# Define tool for OpenAI function calling
tools = [
    {
        "type": "function", 
        "function": {        
            "name": "get_weather",
            "description": "Get weather information for a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state e.g. Mumbai, India"
                    }
                },
                "required": ["location"]
            }
        }
    }
]

# Start conversation
user_message = "What is the temperature of Delhi?"
messages = [{"role": "user", "content": user_message}]

# First API call - AI decides if it needs to call a function
response = client.chat.completions.create(
    model="gpt-4.1-nano",
    messages=messages,
    max_tokens=150,
    temperature=0.7,
    tools=tools,
)

# Check if AI wants to call a function
if response.choices[0].message.tool_calls:
    # Extract function call details
    tool_call = response.choices[0].message.tool_calls[0]
    function_args = json.loads(tool_call.function.arguments)
    
    # Call the actual weather function
    weather_result = get_weather(function_args["location"])
    
    # Add AI's function call request to messages
    messages.append(response.choices[0].message)
    
    # Add function result to messages
    messages.append({
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": str(weather_result)
    })
    
    # Second API call - AI responds with the weather information
    final_response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=messages,
        max_tokens=150,
        temperature=0.7,
    )
    
    print(final_response.choices[0].message.content)
else:
    # No function call needed, print direct response
    print(response.choices[0].message.content)

BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_s8LPkz7d8u3i2tgbyitnhvVt", 'type': 'invalid_request_error', 'param': 'messages', 'code': None}}

# How to create tools in OpenAI

tools = [
    {
        "type": "function",
        "function": {
            "name": "your_function_name",  # Must match your Python function name
            "description": "Clear description of what this function does",
            "parameters": {
                "type": "object",
                "properties": {
                    "param1": {
                        "type": "string",  # or "number", "boolean", "array"
                        "description": "What is param1 for"
                    },
                    "param2": {
                        "type": "number",
                        "description": "What is param2 for"
                    }
                },
                "required": ["param1"]  # Which parameters are mandatory
            }
        }
    }
]

# Flow to make any tool

Define Your Function (Python Function)
Create Tool Schema (OpenAI Format) [recomended to check doc as they keep on updating the tool format]
First API Call (Decision Making)
Check Response Type    if (response.choices[0].message.tool_calls exsist then function/tool call)/is not then direct answer
Execute Function (If Requested response.choices[0].message.tool_calls)
Update Conversation
Second API Call (Final Response)

In [2]:
# Import required libraries
from openai import OpenAI
from dotenv import load_dotenv
import os
import requests
import json

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))  # Fixed: OPENAI_API_KEY instead of OPEN_API_KEY

# Weather function using RapidAPI
def get_weather(location):
    url = "https://ai-weather-by-meteosource.p.rapidapi.com/find_places"
    querystring = {"text": location, "language": "en"}
    headers = {
        "x-rapidapi-key": "cd63083a33msha2e5829aa1a8e78p183c42jsn92196e6a83c6",
        "x-rapidapi-host": "ai-weather-by-meteosource.p.rapidapi.com"
    }
    
    try:
        response = requests.get(url, headers=headers, params=querystring)
        response.raise_for_status()  # Raise an exception for bad status codes
        return response.json()
    except requests.exceptions.RequestException as e:
        return {"error": f"Failed to fetch weather data: {str(e)}"}

# Define tool for OpenAI function calling
tools = [
    {
        "type": "function", 
        "function": {
            "name": "get_weather",
            "description": "Get weather information for a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state e.g. Mumbai, India"
                    }
                },
                "required": ["location"]
            }
        }
    }
]

# Start conversation
user_message = "What is the temperature of Delhi?"
messages = [{"role": "user", "content": user_message}]

try:
    # First API call - AI decides if it needs to call a function
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # Fixed: Use a valid model name
        messages=messages,
        max_tokens=150,
        temperature=0.7,
        tools=tools,
        tool_choice="auto"  # Added: Explicitly set tool choice
    )

    # Check if AI wants to call a function
    if response.choices[0].message.tool_calls:
        # Add AI's response with tool calls to messages
        messages.append(response.choices[0].message)
        
        # Process each tool call
        for tool_call in response.choices[0].message.tool_calls:
            # Extract function call details
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            print(f"Calling function: {function_name} with args: {function_args}")
            
            # Call the actual weather function
            if function_name == "get_weather":
                weather_result = get_weather(function_args["location"])
            else:
                weather_result = {"error": "Unknown function"}
            
            # Add function result to messages
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": str(weather_result)
            })
        
        # Second API call - AI responds with the weather information
        final_response = client.chat.completions.create(
            model="gpt-4o-mini",  # Fixed: Use a valid model name
            messages=messages,
            max_tokens=150,
            temperature=0.7,
        )
        
        print(final_response.choices[0].message.content)
    else:
        # No function call needed, print direct response
        print(response.choices[0].message.content)

except Exception as e:
    print(f"An error occurred: {str(e)}")
    print("Please check your API key and model name.")

Calling function: get_weather with args: {'location': 'Delhi, India'}
I currently do not have real-time weather data. To find the current temperature in Delhi, I recommend checking a reliable weather website or app.


In [10]:
# Import required libraries
from openai import OpenAI
from dotenv import load_dotenv
import os
import requests
import json

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))  

# Weather function using RapidAPI
def get_weather(location):
    find_url = "https://ai-weather-by-meteosource.p.rapidapi.com/find_places"
    find_params = {"text": location, "language": "en"}
    headers = {
        "x-rapidapi-key": "cd63083a33msha2e5829aa1a8e78p183c42jsn92196e6a83c6",
        "x-rapidapi-host": "ai-weather-by-meteosource.p.rapidapi.com"
    }
    
    try:
        find_response = requests.get(find_url, headers=headers, params=find_params)
        find_response.raise_for_status()
        places_data = find_response.json()
        
        if not places_data or len(places_data) == 0:
            return {"error": f"Location '{location}' not found"}
        place = places_data[0]
        place_id = place.get('place_id')
        
        if not place_id:
            return {"error": "Could not get place ID for location"}
        
        weather_url = "https://ai-weather-by-meteosource.p.rapidapi.com/current"
        weather_params = {"place_id": place_id, "timezone": "auto", "language": "en", "units": "auto"}
        
        weather_response = requests.get(weather_url, headers=headers, params=weather_params)
        weather_response.raise_for_status()
        weather_data = weather_response.json()
        
        current = weather_data.get('current', {})
        return {
            "location": location,
            "temperature": current.get('temperature'),
            "feels_like": current.get('feels_like'),
            "humidity": current.get('humidity'),
            "wind_speed": current.get('wind', {}).get('speed'),
            "weather_summary": current.get('summary'),
            "icon": current.get('icon')
        }
        
    except requests.exceptions.RequestException as e:
        return {"error": f"Failed to fetch weather data: {str(e)}"}
    except json.JSONDecodeError as e:
        return {"error": f"Invalid response format: {str(e)}"}
    except Exception as e:
        return {"error": f"Unexpected error: {str(e)}"}

tools = [
    {
        "type": "function", 
        "function": {
            "name": "get_weather",
            "description": "Get weather information for a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state e.g. Mumbai, India"
                    }
                },
                "required": ["location"]
            }
        }
    }
]

# Start conversation
user_message = "What is the wind speed of Delhi?"
messages = [{"role": "user", "content": user_message}]

try:
    # First API call - AI decides if it needs to call a function
    response = client.chat.completions.create(
        model="gpt-4o-mini", 
        messages=messages,
        max_tokens=150,
        temperature=0.7,
        tools=tools,
        tool_choice="auto" 
    )
    # Check if AI wants to call a function
    if response.choices[0].message.tool_calls:
        messages.append(response.choices[0].message)
        for tool_call in response.choices[0].message.tool_calls:

            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            print(f"Calling function: {function_name} with args: {function_args}")
            
            if function_name == "get_weather":
                weather_result = get_weather(function_args["location"])
            else:
                weather_result = {"error": "Unknown function"}
            
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": str(weather_result)
            })
        
        final_response = client.chat.completions.create(
            model="gpt-4o-mini",  
            messages=messages,
            max_tokens=25,
            temperature=0.6,
        )
        
        print(final_response.choices[0].message.content)
    else:
        print(response.choices[0].message.content)

except Exception as e:
    print(f"An error occurred: {str(e)}")
    print("Please check your API key and model name.")

Calling function: get_weather with args: {'location': 'Delhi, India'}
The current wind speed in Delhi is 3.4 m/s. The weather is mostly cloudy, with a temperature of 30.5°C and a humidity level of 36%.
